This notebook is a "meta-notebook" in that it generates other notebooks to actually get run.

The substitution rules for template notebooks are:

* The template must start with "TEMPLATE", which will be removed in the output name.
* All outputs in the templates are removed
* Any text in the *first* code cell that is of the pattern "{T{SOMETHING}" will be replaced by whatever the tempate has for the key SOMETHING.
* Any text in the notebook filename that matches a key will be replaced

In [1]:
import re
from pathlib import Path

import nbformat

from astroquery.mast import Observations

In [2]:
PROPOSAL_ID = 2609

In [3]:
def apply_to_templates(templates, template_mapping):
    outputs = {}
    for tname, template in templates.items():
        if not (tname.startswith('TEMPLATE') and tname.endswith('.ipynb')):
            raise ValueError('not a TEMPLATE notebook!')
            
        nb = nbformat.NotebookNode(template)  # copy to not overwrite the template

        cells = nb['cells']
        for cell in cells:
            if 'outputs' in cell:
                cell['outputs'] = []
        src = cells[0]['source']
        for k, v in template_mapping.items():
            src = src.replace("{T{"+k+"}", v)
        cells[0]['source'] = src

        nbname = tname.removeprefix('TEMPLATE')
        for k, v in template_mapping.items(): 
            nbname = nbname.replace(k, v)
            
        outputs[nbname] = nb
    return outputs

In [4]:
templates = {templatepath.name:nbformat.read(templatepath.open(), 4)
             for templatepath in list(Path('.').glob('TEMPLATE*.ipynb'))}

# Select the slits to run

In [5]:
obses = Observations.query_criteria(proposal_id=PROPOSAL_ID)
specobses = obses[(obses['calib_level']==3)&(obses['dataproduct_type'] == 'spectrum')]
specobses

intentType,obs_collection,provenance_name,instrument_name,project,filters,wavelength_region,target_name,target_classification,obs_id,s_ra,s_dec,dataproduct_type,proposal_pi,calib_level,t_min,t_max,t_exptime,em_min,em_max,obs_title,t_obs_release,proposal_id,proposal_type,sequence_number,s_region,jpegURL,dataURL,dataRights,mtFlag,srcDen,obsid,objID
str7,str4,str7,str12,str4,str12,str8,str27,str23,str40,float64,float64,str8,str14,int64,float64,float64,float64,float64,float64,str60,float64,str4,str2,int64,str115,str66,str67,str16,bool,float64,str9,str9
science,JWST,CALJWST,NIRSPEC/MSA,JWST,F100LP;G140H,Infrared,IC166-NEWCATALOG-GAIAEDR3,--,jw02609-o007_s01035_nirspec_f100lp-g140h,28.09702,61.86512777777778,spectrum,"Nidever, David",3,59891.81792711713,59891.855679699074,291.778,810.0,5000.0,Searching for the Alpha-Abundance Bimodality in the M31 Disk,60256.96119201,2609,GO,--,POLYGON 28.074579882 61.877969446 28.075552087 61.877969446 28.075552087 61.877667174 28.074579882 61.877667174,mast:JWST/product/jw02609-o007_s01035_nirspec_f100lp-g140h_cal.jpg,mast:JWST/product/jw02609-o007_s01035_nirspec_f100lp-g140h_s2d.fits,PUBLIC,False,nan,100544641,390754414
science,JWST,CALJWST,NIRSPEC/MSA,JWST,F100LP;G140H,Infrared,IC166-NEWCATALOG-GAIAEDR3,--,jw02609-o007_s00970_nirspec_f100lp-g140h,28.09702,61.86512777777778,spectrum,"Nidever, David",3,59891.81792711713,59891.855679699074,291.778,810.0,5000.0,Searching for the Alpha-Abundance Bimodality in the M31 Disk,60256.96119201,2609,GO,--,POLYGON 28.067236965 61.887788335 28.068213199 61.887788335 28.068213199 61.887483777 28.067236965 61.887483777,mast:JWST/product/jw02609-o007_s00970_nirspec_f100lp-g140h_cal.jpg,mast:JWST/product/jw02609-o007_s00970_nirspec_f100lp-g140h_s2d.fits,PUBLIC,False,nan,100544655,390754416
science,JWST,CALJWST,NIRSPEC/MSA,JWST,F100LP;G140H,Infrared,IC166-NEWCATALOG-GAIAEDR3,--,jw02609-o007_s06046_nirspec_f100lp-g140h,28.09702,61.86512777777778,spectrum,"Nidever, David",3,59891.81792711713,59891.855679699074,291.778,810.0,5000.0,Searching for the Alpha-Abundance Bimodality in the M31 Disk,60256.96119201,2609,GO,--,POLYGON 28.188633601 61.87839925 28.189616714 61.87839925 28.189616714 61.87808739 28.188633601 61.87808739,mast:JWST/product/jw02609-o007_s06046_nirspec_f100lp-g140h_cal.jpg,mast:JWST/product/jw02609-o007_s06046_nirspec_f100lp-g140h_s2d.fits,PUBLIC,False,nan,100544665,390754417
science,JWST,CALJWST,NIRSPEC/MSA,JWST,F100LP;G140H,Infrared,IC166-NEWCATALOG-GAIAEDR3,--,jw02609-o007_s01040_nirspec_f100lp-g140h,28.09702,61.86512777777778,spectrum,"Nidever, David",3,59891.81792711713,59891.855679699074,291.778,810.0,5000.0,Searching for the Alpha-Abundance Bimodality in the M31 Disk,60256.96119201,2609,GO,--,POLYGON 28.080246846 61.872897263 28.081216217 61.872897263 28.081216217 61.872596172 28.080246846 61.872596172,mast:JWST/product/jw02609-o007_s01040_nirspec_f100lp-g140h_cal.jpg,mast:JWST/product/jw02609-o007_s01040_nirspec_f100lp-g140h_s2d.fits,PUBLIC,False,nan,100544677,390754418
science,JWST,CALJWST,NIRSPEC/MSA,JWST,F100LP;G140H,Infrared,IC166-NEWCATALOG-GAIAEDR3,--,jw02609-o007_s05379_nirspec_f100lp-g140h,28.09702,61.86512777777778,spectrum,"Nidever, David",3,59891.81792711713,59891.855679699074,291.778,810.0,5000.0,Searching for the Alpha-Abundance Bimodality in the M31 Disk,60256.96119201,2609,GO,--,POLYGON 28.204804994 61.888792613 28.205787251 61.888792613 28.205787251 61.888479405 28.204804994 61.888479405,mast:JWST/product/jw02609-o007_s05379_nirspec_f100lp-g140h_cal.jpg,mast:JWST/product/jw02609-o007_s05379_nirspec_f100lp-g140h_s2d.fits,PUBLIC,False,nan,100544688,390754420
science,JWST,CALJWST,NIRSPEC/MSA,JWST,F100LP;G140H,Infrared,IC166-NEWCATALOG-GAIAEDR3,--,jw02609-o007_s05185_nirspec_f100lp-g140h,28.09702,61.86512777777778,spectrum,"Nidever, David",3,59891.81792711713,59891.855679699074,291.778,810.0,5000.0,Searching for the Alpha-Abundance Bimodality in the M31 Disk,60256.96119201,2609,GO,--,POLYGON 28.169943836 61.862103826 28.170927537 61

In [6]:
target_name = 'NGC6791-NEWCATALOG-GAIAEDR3'
specnames = []
for row in specobses[specobses['target_name'] == target_name]:
    m = re.match('jw.*_(s.*?)_.*', row['obs_id'])
    if m is not None:
        specnames.append((target_name, m.group(1)))

# Generate the notebooks

In [7]:
nbpaths_to_exec = []
for targetname, slitname in specnames:
    outputs = apply_to_templates(templates,
                                   {'DLPATHNAME': 'mastDownload/flat/',
                                    'REDUXBASENAME': 'miscdata/jwst_outputs/',
                                    'PROPOSALID': str(PROPOSAL_ID),
                                    'SLITNAME': slitname,
                                    'TARGETNAME': targetname,
                                   })
    for k,v in outputs.items():
        outfn = Path('miscdata/jwst_outputs') / k
        print('writing to', outfn)
        nbformat.write(v, outfn)
        nbpaths_to_exec.append(outfn)
    break  # REMOVE WHEN DONE TESTING

writing to miscdata/jwst_outputs/NGC6791-NEWCATALOG-GAIAEDR3_s00066_slit_dl_and_pipeline.ipynb


# Commands to execute the notebooks

In [8]:
for nbp in nbpaths_to_exec:
    print(f'jupyter nbconvert --to notebook --execute {nbp}')

jupyter nbconvert --to notebook --execute miscdata/jwst_outputs/NGC6791-NEWCATALOG-GAIAEDR3_s00066_slit_dl_and_pipeline.ipynb


Or for split into pipeline and non-pipeline ones:

In [9]:
for nbp in nbpaths_to_exec:
    if 'pipeline' in str(nbp):
        print(f'jupyter nbconvert --to notebook --execute {nbp}')

jupyter nbconvert --to notebook --execute miscdata/jwst_outputs/NGC6791-NEWCATALOG-GAIAEDR3_s00066_slit_dl_and_pipeline.ipynb


In [10]:
for nbp in nbpaths_to_exec:
    if 'pipeline' not in str(nbp):
        print(f'jupyter nbconvert --to notebook --execute {nbp}')